Pour demontrer l'API Spark Streaming en action, nous réproduisons l'example "wordCount", disponible dans la documentation de Spark. Pour le faire, nous ouvrirons d'abord une nouvelle fenêtre sur JupyterHub avec un Terminal. En suite, tapez la commande suivante:

<code>ncat -lk 9999</code>

Maintenant, tout ce vous taperez dans cette fenêtre sera visible sur le port 9999. Le code ci-dessous initialize un Spark Streaming Context (la convention est d'appeler la variable <code>ssc</code>) et dit à Spark de monitorer le port 9999.

Spark devrait charger en temps-réel ce que nous tapons dans le Terminal et compter combien de mots nous avons tapé.

In [ ]:
import pyspark
from pyspark.streaming import StreamingContext

#Uncomment the next line to run the code block on jupyter. Keep it commented if copy-pasting into the pyspark shell
#sc = pyspark.SparkContext()

# This tells Spark Streaming to bacth-up the contents of a data stream and "ingest" them every 10 seconds.
ssc = StreamingContext(sc,10)

# Tell spark to listen on port 9999 of our localhost.
lines = ssc.socketTextStream("localhost", 9999)

words = lines.flatMap(lambda line : line.split(" "))

pairs = words.map(lambda word: (word, 1))
wordCount = pairs.reduceByKey(lambda a, b: a + b)

wordCount.pprint()

ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()

Les méthodes disponibles dans l'API Spark Streaming sont très semblables à ceux de l'API RDD. L'example suivat initialise un Spark Streaming Context et dit à Spark de monitorer un répertoire dans notre systeme d'archives. À chaque fois qu'un nouveau fichier est placé dans le répertoire, Spark chargera le contenu en format Texte.

In [ ]:
import pyspark
from pyspark.streaming import StreamingContext

#sc = pyspark.SparkContext()
ssc = StreamingContext(sc,10)

records = ssc.textFileStream("/home/user74/scratch/")

rows = records.map(lambda line : line.split(","))
rows.pprint()

ssc.start()
ssc.awaitTermination()

In [ ]:
ssc.stop()

Tel que nous l'avons vu avant, l'API RDD est super, mais l'API SparkSQL rend la tâche de manipuler des données beaucoup plus facile. Nous pouvons l'utiliser en pointant Spark Streaming vers un DataFrame, qui sera populé avec des données au fur et à mesure qu'ils devienent disponibles dans le Stream:

In [1]:
import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext()

spark = SQLContext(sc)

my_dataframe = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

Ceci dit à Spark de monitorer le port 9999, comme nous l'avons fait dans le premier exemple. La différence est que ici nous disons à Spark que nous voulons stocker le contenu du Stream dans un DataFrame! En suite, nous disons à Spark quelles opérations nous voulons appliquer sur nos données en temps réel.

Dans cet exemple, nous écrirons le contenu du Stream en mémoire et nous l'appelerons "new_dataframe". Nous dirons aussi à Spark de les ajouter au DataFrame au fur et à mesure. Cela est équivalent à créer un Tableau et le populer avec le contenu du Stream rentrant!

In [3]:
my_dataframe.writeStream.format("memory").queryName("new_dataframe").outputMode("append").start()

Vous pouvez vérifier que nous avons un nouveau Tableau "new_dataframe":

In [5]:
spark.tableNames()

['new_dataframe']

Et vous pouvez vous servir du SQL pour explorer le Tableau. Ici nous assumons que le contenu du Stream ce sont des colonnes séparées par des virgules que nous transformons en colonnes d'un Tableau.

In [25]:
spark.sql("SELECT SPLIT(value,',')[0] AS Col1, SPLIT(value,',')[1] as Col2 FROM new_dataframe").collect()

[Row(Col1='this is a test', Col2=None, Col3=None),
 Row(Col1='test', Col2=None, Col3=None),
 Row(Col1='test', Col2=None, Col3=None),
 Row(Col1='test', Col2=None, Col3=None),
 Row(Col1='this', Col2=' is', Col3=' a')]

Nous avons demontré comment Spark Streaming fonctionne en simulant une queue de messages où Spark lit du Texte. Cela est un cas très frequent d'utilisation de Spark Streaming dans la vraie vie. Vous pouvez aussi lire des messages en d'autres formats, tels que le JSON par exemple.

La structure du code pour initialiser un Stream et travailler avec d'autres formats que le Text est très similaire aux examples ci-dessus, il suffit de changer l'option "format"! Pour plus d'infos sur les autres formats: https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#api-using-datasets-and-dataframes

## Exercise 1 - What is the Spread?

In finance, the term **spread** is often used to refer to the difference between two metrics of interest. In the stock market in particular, the **Bid-Ask spread** is one of the many tools used to help inform decision making around a trader's positions in the market. The Bid-Ask spread is simply the difference between the price traders selling a security are asking, **the ask**, and the price traders buying that security are offering to pay, **the bid**. 

A prized piece of information that many firms pay good money to acquire is known as a **BBO** - the **Best Bid-Offer** datset. BBO datasets are simple tabular-formatted collections of data with 3 key columns: a timestamp, the best bid and the best offer for a given security at that exact timestamp. Stock brokers have a fiduciary duty to get the best bid for their clients selling a security and the best offer for their clients buying a security. Hence the usefulness of BBO data... but that is not all! If you have current BBO data, you can also keep an eye on the **spread** on a given security in real-time. Knowing this quantity in real-time can be used in many ways, the simplest of which is as a gauge of the **liquidity** of a security. In general, if the **spread** on a security is small, that suggests there is a hot market for that security (i.e. people are actively buying and selling the seurity). Conversely, if the spread is large, that suggests the market is not really interested in trading that security at that time.

We will not show you how to create your own BBO dataset today, but we will use one to keep an eye on the spread of a certain stock. 

First, let's simulate a real-time feed from a BBO provider. To do this, run the following command on a terminal window:

<code>stdbuf -oL cat data/14081.csv | ncat localhost 9999</code>

Now, use the Structured Streaming approach we've seen before to read the BBO data into a DataFrame and get the **Spread** second-by-second. If you are looking for a challenge, try computing the **average spread** on a minute-by-minute basis.

## Exercise 2 - Anomaly Detection

Now let's look at another application of Spark: anomaly detection. In this problem, a fictional Utility Company started using Machine Learning to determine the price it charges its customers depending on the hour of the day. You believe this move on the part of the Utility Company causing you to pay way too much for electricity and have decided to put together a dossier exposing how their algorithm is out of whack. One way of exposing the weakness of their algorithm would be to catch anomalies in the price they are charging. The company themselves defines an anomaly as "**a 2 standard deviation or larger increase over the average price for the same hour over the past two weeks.**"

Use the SparkSQL API to read the <code>utility.csv</code> and find instances of anomalous pricing.